<a href="https://colab.research.google.com/github/Open-Reaction-Database/ord-schema/blob/main/examples/2_Nielsen_Deoxyfluorination_Screen/example_nielsen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deoxyfluorination with Sulfonyl Fluorides: Navigating Reaction Space with Machine Learning

DOI: 10.1021/jacs.8b01523

J. Am. Chem. Soc. 2018, 140, 5004−5008

## Defining protos for reaction data in Figure 1

Colab set-up: install schema

In [1]:
try:
    import ord_schema
    import rdkit
except:
    import sys
    !wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
    !time bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
    !time conda install -q -y -c rdkit rdkit
    !time conda install -q -y -c anaconda protobuf
    !git clone https://github.com/Open-Reaction-Database/ord-schema.git
    %cd ord-schema
    !python setup.py install
    sys.path.append('/usr/local/lib/python3.7/site-packages/')

Import schema and helper functions

In [2]:
from datetime import datetime
from ord_schema.proto import reaction_pb2
from ord_schema.proto import dataset_pb2
from ord_schema.units import UnitResolver
from ord_schema import validations
from ord_schema import message_helpers

unit_resolver = UnitResolver()

# Define exemplary reaction

This will be a single prototypical reaction from Table 1. We will copy it and use it as a template for filling out the full results table.

In [3]:
reaction = reaction_pb2.Reaction()
reaction.identifiers.add(value=r'deoxyfluorination', type='NAME')
reaction

identifiers {
  type: NAME
  value: "deoxyfluorination"
}

Define reaction inputs
- The first input is a pre-mixed stock solution of an alcohol in THF (1a for the prototype reaction)
- The second input is a pre-mixed stock solution of a sulfonyl fluoride (4-chlorobenzenesulfonyl fluoride for the prototype reaction)
- The third addition is a pure base (DBU for the prototype reaction)

In [4]:
# Input 1a: 0.1 mmol alcohol in 125 uL THF
input_1a = reaction.inputs['alcohol in THF']
input_1a.addition_order = 1

solute = input_1a.components.add()
solvent = input_1a.components.add()

solute.reaction_role = reaction_pb2.ReactionRole.REACTANT 
solute.identifiers.add(value=r'c1ccccc1CCC(O)C', type='SMILES')
solute.amount.moles.CopyFrom(unit_resolver.resolve('0.1 mmol'))
solute.preparations.add().type = reaction_pb2.CompoundPreparation.PreparationType.NONE
solute.is_limiting = True

solvent.reaction_role = reaction_pb2.ReactionRole.SOLVENT
solvent.identifiers.add(value=r'THF', type='NAME')
solvent.identifiers.add(value=r'C1CCCO1', type='SMILES')
solvent.amount.volume.CopyFrom(unit_resolver.resolve('125 uL'))
solvent.preparations.add().type = reaction_pb2.CompoundPreparation.PreparationType.DRIED

## Note: more lengthy way to specify volume w/o unit resolver
# solvent.volume.value = 125
# solvent.volume.units = reaction_pb2.Volume.MICROLITER

In [5]:
# Input sf: 0.11 mmol sulfonyl fluoride in 125 uL THF
input_sf = reaction.inputs['sulfonyl fluoride']
input_sf.addition_order = 2

solute = input_sf.components.add()
solvent = input_sf.components.add()

solute.reaction_role = reaction_pb2.ReactionRole.REACTANT 
solute.identifiers.add(value=r'4-chlorobenzenesulfonyl fluoride', type='NAME')
solute.identifiers.add(value=r'Clc1ccc(S(=O)(=O)F)cc1', type='SMILES')
solute.amount.moles.CopyFrom(unit_resolver.resolve('0.11 mmol'))
solute.preparations.add().type = reaction_pb2.CompoundPreparation.PreparationType.SYNTHESIZED

solvent.reaction_role = reaction_pb2.ReactionRole.SOLVENT
solvent.identifiers.add(value=r'THF', type='NAME')
solvent.identifiers.add(value=r'C1CCCO1', type='SMILES')
solvent.amount.volume.CopyFrom(unit_resolver.resolve('125 uL'))
solvent.preparations.add().type = reaction_pb2.CompoundPreparation.PreparationType.DRIED

In [6]:
# Input base: 0.15 mmol pure
input_base = reaction.inputs['base']
input_base.addition_order = 3
base = input_base.components.add()

base.reaction_role = reaction_pb2.ReactionRole.REAGENT
base.identifiers.add(value=r'N\2=C1\N(CCCCC1)CCC/2', type='SMILES')
base.amount.moles.CopyFrom(unit_resolver.resolve('0.15 mmol'))
base.preparations.add().type = reaction_pb2.CompoundPreparation.PreparationType.NONE
base.source.vendor = 'Sigma-Millipore'

Define reaction setup & conditions

In [7]:
# Reactions performed in 1 mL sealed glass vial
reaction.setup.vessel.CopyFrom(
    reaction_pb2.Vessel(
        type=dict(type='VIAL'), 
        material=dict(type='GLASS'),
        volume=unit_resolver.resolve('1 mL')
    )
)
reaction.setup.vessel.attachments.add(type='CAP')
reaction.setup.is_automated = False

In [8]:
# No temperature control = ambient conitions
t_conds = reaction.conditions.temperature
t_conds.control.type = t_conds.TemperatureControl.AMBIENT

In [9]:
# Vials were sealed under ambient conditions
p_conds = reaction.conditions.pressure
p_conds.control.type = p_conds.PressureControl.SEALED
p_conds.atmosphere.type = p_conds.Atmosphere.AIR

In [10]:
# Vials contained stir bars at 600 rpm
s_conds = reaction.conditions.stirring
s_conds.method.type = s_conds.StirringMethod.STIR_BAR
s_conds.rate.type = s_conds.StirringRate.HIGH # qualitative
s_conds.rate.rpm = 600

No additional safety notes or observations

In [11]:
# No safety notes
# reaction.notes.safety_notes = ''

# No reaction observations
# observation = reaction.observations.add()
# observation.time = 
# observation.comment = 

In [12]:
# Only workup is addition of NMR standard
workup = reaction.workups.add(type='ADDITION')

solute = workup.input.components.add()
solvent = workup.input.components.add()

solute.reaction_role = reaction_pb2.ReactionRole.INTERNAL_STANDARD 
solute.identifiers.add(value=r'C1=CC=C2C(=C1)C=CC=C2F', type='SMILES')
solute.identifiers.add(value=r'1-fluoronaphthalene', type='NAME')
solute.amount.moles.CopyFrom(unit_resolver.resolve('0.1 mmol'))

solvent.reaction_role = reaction_pb2.ReactionRole.WORKUP
solvent.identifiers.add(value=r'chloroform', type='NAME')
solvent.identifiers.add(value=r'ClC(Cl)Cl', type='SMILES')
solvent.amount.volume.CopyFrom(unit_resolver.resolve('250 uL'))

After 48 hours, the crude reaction mixture is characterized by 19F NMR to estimate the yield

This represents one "outcome" with one characterized "product"

In [13]:
outcome = reaction.outcomes.add()
outcome.reaction_time.CopyFrom(unit_resolver.resolve('48 hrs'))

# Analyses: 19F NMR
outcome.analyses['19f nmr of crude'].type = reaction_pb2.Analysis.NMR_OTHER
outcome.analyses['19f nmr of crude'].details = ('19F NMR using 1 equiv 1-fluoro'
  'naphthalene in 250 uL chloroform as internal standard')
outcome.analyses['19f nmr of crude'].instrument_manufacturer = 'Bruker'

# Define product identity
prod_2a = outcome.products.add(is_desired_product=True)
prod_2a.identifiers.add(type='SMILES', value='c1ccccc1CCC(F)C')
prod_2a.reaction_role = reaction_pb2.ReactionRole.PRODUCT

# Define product yield from Figure 1 in the main test
# Note: described in SI, 4.8% is the variance of the yield including variance
# in the experiment itself. NMR yields itself has precision of <= 2%
# accoring to SI Section II.
prod_2a.measurements.add(type='YIELD', analysis_key='19f nmr of crude',
                         percentage=dict(value=40, precision=4.8), uses_internal_standard=True)
# The "19f nmr of crude" analysis was used to confirm both identity and yield
prod_2a.measurements.add(type='IDENTITY', analysis_key='19f nmr of crude')

# Reaction provenance
reaction.provenance.city = r'Princeton, NJ'
reaction.provenance.doi = r'10.1021/jacs.8b01523'
reaction.provenance.publication_url = r'https://pubs.acs.org/doi/10.1021/jacs.8b01523'
reaction.provenance.record_created.time.value = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
reaction.provenance.record_created.person.CopyFrom(reaction_pb2.Person(
    name='Connor W. Coley', organization='MIT', orcid='0000-0002-8271-8723', email='ccoley@mit.edu'
))

Validate and examine this final prototypical reaction entry -- note that this is just a single entry from the results table in Figure 1

In [14]:
validations.validate_message(reaction)
print(reaction)

identifiers {
  type: NAME
  value: "deoxyfluorination"
}
inputs {
  key: "alcohol in THF"
  value {
    components {
      identifiers {
        type: SMILES
        value: "c1ccccc1CCC(O)C"
      }
      amount {
        moles {
          value: 0.10000000149011612
          units: MILLIMOLE
        }
      }
      reaction_role: REACTANT
      is_limiting: true
      preparations {
        type: NONE
      }
    }
    components {
      identifiers {
        type: NAME
        value: "THF"
      }
      identifiers {
        type: SMILES
        value: "C1CCCO1"
      }
      amount {
        volume {
          value: 125.0
          units: MICROLITER
        }
      }
      reaction_role: SOLVENT
      preparations {
        type: DRIED
      }
    }
    addition_order: 1
  }
}
inputs {
  key: "base"
  value {
    components {
      identifiers {
        type: SMILES
        value: "N\\2=C1\\N(CCCCC1)CCC/2"
      }
      amount {
        moles {
          value: 0.15000000596046448

# Define full set of compounds and conditions

We will make extensive use of the ```message_helpers.build_compound``` helper function and define our own custom helper function to use the prototypical reaction example and replace only select entries

In [15]:
# Defining all major reactants, reagents, and products used in screen

compounds = {
  # Reactants
  '1a': message_helpers.build_compound(r'c1ccccc1CCC(O)C'),
  '2a': message_helpers.build_compound(r'c1ccccc1CCC(F)C'),
  '1b': message_helpers.build_compound(r'c1ccccc1-c2ccc(CO)cc2'),
  '2b': message_helpers.build_compound(r'c1ccccc1-c2ccc(CF)cc2'),
  '1c': message_helpers.build_compound(r'O[C@@H]1C[C@H](OCC2=CC=CC=C2)C1'),
  '2c': message_helpers.build_compound(r'F[C@@H]1C[C@H](OCC2=CC=CC=C2)C1'),
  '1d': message_helpers.build_compound(r'O[C@H]1C[C@@H](C(OC)=O)N(C(OC(C)(C)C)=O)C1'),
  '2d': message_helpers.build_compound(r'F[C@H]1C[C@@H](C(OC)=O)N(C(OC(C)(C)C)=O)C1'),
  
  # Sulfonyl fluorides
  '3-Cl': message_helpers.build_compound(r'Clc1ccc(S(=O)(=O)F)cc1'),
  'PyFluor': message_helpers.build_compound(r'O=S(C1=CC=CC=N1)(F)=O', name='PyFluor'),
  '3-CF3': message_helpers.build_compound(r'O=S(C1=CC=C(C(F)(F)F)C=C1)(F)=O'),
  '3-NO2': message_helpers.build_compound(r'O=S(C1=CC=C([N+]([O-])=O)C=C1)(F)=O'),
  'PBSF': message_helpers.build_compound(r'C(C(C(F)(F)S(=O)(=O)F)(F)F)(C(F)(F)F)(F)F', 
                         vendor='Acros'),
  # Bases
  'DBU': message_helpers.build_compound(r'N\2=C1\N(CCCCC1)CCC/2', name='DBU', 
                           vendor='Millipore-Sigma'),
  'MTBD': message_helpers.build_compound(r'CN1CCCN2C1=NCCC2', name='MTBD',
                            vendor='Sigma-Millipore'),
  'BTMG': message_helpers.build_compound(r'CC(C)(C)N=C(N(C)C)N(C)C', name='BTMG', 
                            vendor='Millipore-Sigma'),
  'BTPP': message_helpers.build_compound(r'CC(C)(C)N=P(N1CCCC1)(N2CCCC2)N3CCCC3', 
                            name='BTPP', vendor='Millipore-Sigma'),
}

In [16]:
# Define yield tables
data = [ # reactant, product, sulfonyl fluoride, base, yield percent
  # 2a
  ('1a', '2a', '3-Cl', 'DBU', 40),
  ('1a', '2a', '3-Cl', 'MTBD', 54),
  ('1a', '2a', '3-Cl', 'BTMG', 41),
  ('1a', '2a', '3-Cl', 'BTPP', 42),
  ('1a', '2a', 'PyFluor', 'DBU', 57),
  ('1a', '2a', 'PyFluor', 'MTBD', 59),
  ('1a', '2a', 'PyFluor', 'BTMG', 49),
  ('1a', '2a', 'PyFluor', 'BTPP', 53),
  ('1a', '2a', '3-CF3', 'DBU', 52),
  ('1a', '2a', '3-CF3', 'MTBD', 69),
  ('1a', '2a', '3-CF3', 'BTMG', 57),
  ('1a', '2a', '3-CF3', 'BTPP', 60),
  ('1a', '2a', '3-NO2', 'DBU', 54),
  ('1a', '2a', '3-NO2', 'MTBD', 63),
  ('1a', '2a', '3-NO2', 'BTMG', 55),
  ('1a', '2a', '3-NO2', 'BTPP', 51),
  ('1a', '2a', 'PBSF', 'DBU', 39),
  ('1a', '2a', 'PBSF', 'MTBD', 60),
  ('1a', '2a', 'PBSF', 'BTMG', 61),
  ('1a', '2a', 'PBSF', 'BTPP', 65),
  # 2b
  ('1b', '2b', '3-Cl', 'DBU', 11),
  ('1b', '2b', '3-Cl', 'MTBD', 36),
  ('1b', '2b', '3-Cl', 'BTMG', 83),
  ('1b', '2b', '3-Cl', 'BTPP', 92),
  ('1b', '2b', 'PyFluor', 'DBU', 12),
  ('1b', '2b', 'PyFluor', 'MTBD', 26),
  ('1b', '2b', 'PyFluor', 'BTMG', 57),
  ('1b', '2b', 'PyFluor', 'BTPP', 77),
  ('1b', '2b', '3-CF3', 'DBU', 17),
  ('1b', '2b', '3-CF3', 'MTBD', 36),
  ('1b', '2b', '3-CF3', 'BTMG', 83),
  ('1b', '2b', '3-CF3', 'BTPP', 99),
  ('1b', '2b', '3-NO2', 'DBU', 21),
  ('1b', '2b', '3-NO2', 'MTBD', 41),
  ('1b', '2b', '3-NO2', 'BTMG', 83),
  ('1b', '2b', '3-NO2', 'BTPP', 91),
  ('1b', '2b', 'PBSF', 'DBU', 23),
  ('1b', '2b', 'PBSF', 'MTBD', 37),
  ('1b', '2b', 'PBSF', 'BTMG', 48),
  ('1b', '2b', 'PBSF', 'BTPP', 68),
  # 2c
  ('1c', '2c', '3-Cl', 'DBU', 1),
  ('1c', '2c', '3-Cl', 'MTBD', 1),
  ('1c', '2c', '3-Cl', 'BTMG', 1),
  ('1c', '2c', '3-Cl', 'BTPP', 3),
  ('1c', '2c', 'PyFluor', 'DBU', 1),
  ('1c', '2c', 'PyFluor', 'MTBD', 2),
  ('1c', '2c', 'PyFluor', 'BTMG', 1),
  ('1c', '2c', 'PyFluor', 'BTPP', 1),
  ('1c', '2c', '3-CF3', 'DBU', 3),
  ('1c', '2c', '3-CF3', 'MTBD', 4),
  ('1c', '2c', '3-CF3', 'BTMG', 5),
  ('1c', '2c', '3-CF3', 'BTPP', 12),
  ('1c', '2c', '3-NO2', 'DBU', 11),
  ('1c', '2c', '3-NO2', 'MTBD', 12),
  ('1c', '2c', '3-NO2', 'BTMG', 12),
  ('1c', '2c', '3-NO2', 'BTPP', 22),
  ('1c', '2c', 'PBSF', 'DBU', 83),
  ('1c', '2c', 'PBSF', 'MTBD', 79),
  ('1c', '2c', 'PBSF', 'BTMG', 89),
  ('1c', '2c', 'PBSF', 'BTPP', 82),
  # 2d
  ('1d', '2d', '3-Cl', 'DBU', 25),
  ('1d', '2d', '3-Cl', 'MTBD', 30),
  ('1d', '2d', '3-Cl', 'BTMG', 29),
  ('1d', '2d', '3-Cl', 'BTPP', 30),
  ('1d', '2d', 'PyFluor', 'DBU', 45),
  ('1d', '2d', 'PyFluor', 'MTBD', 40),
  ('1d', '2d', 'PyFluor', 'BTMG', 39),
  ('1d', '2d', 'PyFluor', 'BTPP', 33),
  ('1d', '2d', '3-CF3', 'DBU', 41),
  ('1d', '2d', '3-CF3', 'MTBD', 47),
  ('1d', '2d', '3-CF3', 'BTMG', 39),
  ('1d', '2d', '3-CF3', 'BTPP', 33),
  ('1d', '2d', '3-NO2', 'DBU', 48),
  ('1d', '2d', '3-NO2', 'MTBD', 48),
  ('1d', '2d', '3-NO2', 'BTMG', 40),
  ('1d', '2d', '3-NO2', 'BTPP', 32),
  ('1d', '2d', 'PBSF', 'DBU', 59),
  ('1d', '2d', 'PBSF', 'MTBD', 74),
  ('1d', '2d', 'PBSF', 'BTMG', 64),
  ('1d', '2d', 'PBSF', 'BTPP', 58),
]

# Define all reactions in Table 1

The only aspects of reaction data that vary are (a) the identity of the solute of the ```'alcohol in THF'``` input, the identity of the solute in the ```'sulfonyl fluoride'``` input, and the identity of the sole component in the ```'base'``` input. The vessel, reaction setup, solvent, timing, temperature, etc. are all constant. The yield, of course, also changes, but the instrument and type of analysis (19F NMR) is constant.

In [17]:
reactions = []
for alc, prod, sf, base, y in data:
  
    # Start with template
    this_condition = reaction_pb2.Reaction()
    this_condition.CopyFrom(reaction)

    # Modify species:
    # - identifiers, vendor, prep ONLY
    def modify(cpd_from, cpd_to):
        del cpd_to.identifiers[:]
        del cpd_to.preparations[:]
        for identifier in cpd_from.identifiers:
            cpd_to.identifiers.add().CopyFrom(identifier)
        if cpd_from.source.vendor:
            cpd_to.source.vendor = cpd_from.source.vendor
        for preparation in cpd_from.preparations:
            cpd_to.preparations.add().CopyFrom(preparation)
  
    modify(compounds[alc], this_condition.inputs['alcohol in THF'].components[0])
    modify(compounds[sf], this_condition.inputs['sulfonyl fluoride'].components[0])
    modify(compounds[base], this_condition.inputs['base'].components[0])
    del this_condition.outcomes[0].products[0].identifiers[:]
    this_condition.outcomes[0].products[0].identifiers.extend(compounds[prod].identifiers)

    # Modify yield
    this_condition.outcomes[0].products[0].measurements[0].percentage.value = y

    # Validate
    validations.validate_message(this_condition)

    # Save
    reactions.append(this_condition)

In [18]:
print(f'Generated {len(reactions)} reactions')

Generated 80 reactions


In [19]:
# Inspect random reaction from this set
reactions[42]

identifiers {
  type: NAME
  value: "deoxyfluorination"
}
inputs {
  key: "alcohol in THF"
  value {
    components {
      identifiers {
        type: SMILES
        value: "O[C@@H]1C[C@H](OCC2=CC=CC=C2)C1"
      }
      amount {
        moles {
          value: 0.10000000149011612
          units: MILLIMOLE
        }
      }
      reaction_role: REACTANT
      is_limiting: true
    }
    components {
      identifiers {
        type: NAME
        value: "THF"
      }
      identifiers {
        type: SMILES
        value: "C1CCCO1"
      }
      amount {
        volume {
          value: 125.0
          units: MICROLITER
        }
      }
      reaction_role: SOLVENT
      preparations {
        type: DRIED
      }
    }
    addition_order: 1
  }
}
inputs {
  key: "base"
  value {
    components {
      identifiers {
        type: SMILES
        value: "CC(C)(C)N=C(N(C)C)N(C)C"
      }
      identifiers {
        type: NAME
        value: "BTMG"
      }
      amount {
        moles {

### Package reactions into a dataset

In [20]:
dataset = dataset_pb2.Dataset(
    name='Deoxyfluorination screen',
    description='Reactions from Figure 1 of DOI: 10.1021/jacs.8b01523',
    reactions=reactions,    
)
# message_helpers.write_message(dataset, 'ord-nielsen-example.pbtxt')

## Prepare for machine learning
Given the list of reactions (defined here or retrieved through other means), we can prepare a DataFrame that summarizes the conditions and outcomes using only the fields that vary.

In [21]:
import pandas as pd

In [22]:
data_dict = {
    'reaction_id': [],
    'alcohol': [],
    'sulfonyl fluoride': [],
    'base': [],
    'product': [],
    'yield': [],
}

for reaction in reactions:
    data_dict['reaction_id'].append(reaction.reaction_id)
    data_dict['alcohol'].append(
        message_helpers.smiles_from_compound(reaction.inputs['alcohol in THF'].components[0])
    )
    data_dict['sulfonyl fluoride'].append(
        message_helpers.smiles_from_compound(reaction.inputs['sulfonyl fluoride'].components[0])
    )
    data_dict['base'].append(
        message_helpers.smiles_from_compound(reaction.inputs['base'].components[0])
    )
    data_dict['product'].append(
        message_helpers.smiles_from_compound(reaction.outcomes[0].products[0])
    )
    data_dict['yield'].append(reaction.outcomes[0].products[0].measurements[0].percentage.value)
    
pd.DataFrame.from_dict(data_dict)

,reaction_id,alcohol,sulfonyl fluoride,base,product,yield
0,,c1ccccc1CCC(O)C,Clc1ccc(S(=O)(=O)F)cc1,N\2=C1\N(CCCCC1)CCC/2,c1ccccc1CCC(F)C,40.0
1,,c1ccccc1CCC(O)C,Clc1ccc(S(=O)(=O)F)cc1,CN1CCCN2C1=NCCC2,c1ccccc1CCC(F)C,54.0
2,,c1ccccc1CCC(O)C,Clc1ccc(S(=O)(=O)F)cc1,CC(C)(C)N=C(N(C)C)N(C)C,c1ccccc1CCC(F)C,41.0
3,,c1ccccc1CCC(O)C,Clc1ccc(S(=O)(=O)F)cc1,CC(C)(C)N=P(N1CCCC1)(N2CCCC2)N3CCCC3,c1ccccc1CCC(F)C,42.0
4,,c1ccccc1CCC(O)C,O=S(C1=CC=CC=N1)(F)=O,N\2=C1\N(CCCCC1)CCC/2,c1ccccc1CCC(F)C,57.0
...,...,...,...,...,...,...
75,,O[C@H]1C[C@@H](C(OC)=O)N(C(OC(C)(C)C)=O)C1,O=S(C1=CC=C([N+]([O-])=O)C=C1)(F)=O,CC(C)(C)N=P(N1CCCC1)(N2CCCC2)N3CCCC3,F[C@H]1C[C@@H](C(OC)=O)N(C(OC(C)(C)C)=O)C1,32.0
76,,O[C@H]1C[C@@H](C(OC)=O)N(C(OC(C)(C)C)=O)C1,C(C(C(F)(F)S(=O)(=O)F)(F)F)(C(F)(F)F)(F)F,N\2=C1\N(CCCCC1)CCC/2,F[C@H]1C[C@@H](C(OC)=O)N(C(OC(C)(C)C)=O)C1,59.0
77,,O[C@H]1C[C@@H](C(OC)=O)N(C(OC(C)(C)C)=O)C1,C(C(C(F)(F)S(=O)(=O)F)(F)F)(C(F)(F)F)(F)F,CN1CCCN2C1=NCCC2,F[C@H]1C[C@@H](C(OC)=O)N(C(OC(C)(C)C)=O)C1,74.0
78,,O[C@H]1C[C@@H](C(OC)=O)N(C(OC(C)(C)C)=O)C1,C(C(C(F)(F)S(=O)(=O)F)(F)F)(C(F)(F)F)(F)F,CC(C)(C)N=C(N(C)C)N(C)C,F[C@H]1C[C@@H](C(OC)=O)N(C(OC(C)(C)C)=O)C1,64.0


# Text description (work in progress)

In [23]:
from ord_schema.visualization import generate_text

In [24]:
generate_text.generate_text(reactions[1])

'To a 1 mL glass vial was added (1) 0.1 mmol as a limiting reactant + 125 μL THF as a solvent (dried); (2) 0.11 mmol as a reactant + 125 μL THF as a solvent (dried); and (3) 0.15 mmol MTBD as a reagent (purchased from Sigma-Millipore). The reaction was run under ambient temperature conditions. The reaction was run under air after fully sealing the reaction vessel. The reaction mixture was stirred at a high rate (600 rpm) using a stir bar. The workup procedure consisted of (1) <WORKUP_STEP> The reaction was analyzed after 48 h by NMR (other) (19F NMR using 1 equiv 1-fluoronaphthalene in 250 uL chloroform as internal standard)., the desired product, was observed with a yield of 54.000% (± 4.800%)..'